In [1]:
import requests
import time
import base64
import pandas as pd
import os
import time
import operator
from json import JSONDecoder
from tqdm import tqdm

In [2]:
def get_max_emotion(emotions):
    assert isinstance(emotions, dict)
    max_emotion = max(emotions.items(), key=operator.itemgetter(1))[0]
    return max_emotion

def get_beauty_score(beauties):
    """return: (female_score, male_score)"""
    assert isinstance(beauties, dict)
    return tuple(beauties.values())

def get_gender(gender_dict):
    assert isinstance(gender_dict, dict)
    return gender_dict['value']

def get_age(age_dict):
    assert isinstance(age_dict, dict)
    return age_dict['value']

def get_eye_status(eye_status_dict):
    """return: (left_eye_status, right_eye_status)"""
    assert isinstance(eye_status_dict, dict)
    return max(eye_status_dict['left_eye_status'].items(), key=operator.itemgetter(1))[0], \
            max(eye_status_dict['right_eye_status'].items(), key=operator.itemgetter(1))[0]

def get_face_quality(face_quality_dict):
    """threshold': 70.1"""
    return face_quality_dict['value']

def get_ethnicity(ethnicity_ditc):    
    return ethnicity_ditc['value']

In [3]:
dir_path = '/home/nvd/Documents/biometrics/datasets/curated_face_datasets/gender_dataset/raw_data/F'
database = "gender_dataset_F"
df = pd.DataFrame(columns=['Emotion', 'Female_Beauty', 'Male_Beauty', 'Gender', 'Age', 'Left_Eye_Status', 'Right_Eye_Status', 'Face_Quality', 'Ethnicity'])

In [4]:
df_females_orig = pd.read_csv('gender_females.csv')

In [5]:
faceplusplus_api = 'https://api-us.faceplusplus.com/facepp/v3/detect'
key = 'oy6XkMRYm7-GO4qnNArBT0PawBqocSv_'
secret = 'o6YdzzCDKHQD0aqO4idwcSonYT_2o4-R'

In [6]:
data = {}
data['api_key'] = key
data['api_secret'] = secret
data['return_attributes'] = 'gender,age,facequality,eyestatus,emotion,ethnicity,beauty'
rows = {'full_path': [],
        'database': [],
        'emotion': [],
        'female_beauty': [],
        'male_beauty': [],
        'gender': [],
        'age': [],
        'left_eye_status': [],
        'right_eye_status': [],
        'face_quality': [],
        'ethnicity': []}

In [ ]:
df_females_list = df_females_orig.iloc[:]['full_path'].tolist()

images = os.listdir(dir_path)
images_left_to_label = images.copy()

for index, file in tqdm(enumerate(images)):
    # Load image.
    filepath = os.path.join(dir_path, file)
    
    if filepath in df_females_list:
        #print('Exists')
        continue
    
    #print("\nFilepath: ", filepath)
    assert os.path.isfile(filepath), "File does not exist."
    files= {'image_file': open(filepath, 'rb')}
    # Make post.
    response = requests.post(url=faceplusplus_api,
                             data=data,
                             files=files)
    # Parse response into json.
    req_con = response.content.decode('utf-8')
    req_dict = JSONDecoder().decode(req_con)
    #print(req_dict)
    # If no faces are found.
    if len(req_dict['faces'])==0:
        continue
    # Create dataframe row.
    rows['full_path'].append(os.path.join(dir_path, file))
    rows['database'].append(database)
    rows['emotion'].append(get_max_emotion(req_dict['faces'][0]['attributes']['emotion']))
    female_beauty, male_beauty = get_beauty_score(req_dict['faces'][0]['attributes']['beauty'])
    rows['female_beauty'].append(female_beauty)
    rows['male_beauty'].append(male_beauty)                               
    rows['gender'].append(get_gender(req_dict['faces'][0]['attributes']['gender']))
    rows['age'].append(get_age(req_dict['faces'][0]['attributes']['age']))
    left_eye, right_eye = get_eye_status(req_dict['faces'][0]['attributes']['eyestatus'])
    rows['left_eye_status'].append(left_eye)
    rows['right_eye_status'].append(right_eye)
    rows['face_quality'].append(get_face_quality( req_dict['faces'][0]['attributes']['facequality']))
    rows['ethnicity'].append(get_ethnicity(req_dict['faces'][0]['attributes']['ethnicity']))
    time.sleep(1)
    images_left_to_label.remove(file)
    
    if index % 10 == 0:
        df_row_it = pd.DataFrame.from_dict(rows)
        df_row_it.to_csv("gender_females.csv", mode='a', header=False, index=False)
        
        rows = {'full_path': [],
        'database': [],
        'emotion': [],
        'female_beauty': [],
        'male_beauty': [],
        'gender': [],
        'age': [],
        'left_eye_status': [],
        'right_eye_status': [],
        'face_quality': [],
        'ethnicity': []}

41050it [5:57:20,  1.91it/s]

In [ ]:
df = pd.DataFrame.from_dict(rows)

In [ ]:
#df.to_csv("gender_females.csv", index=False)
df.to_csv("gender_females.csv", mode='a', header=False, index=False)

In [ ]:
test_df = pd.read_csv("gender_females.csv")

In [ ]:
test_df.head(15)

In [ ]:
test_df.shape

In [ ]:
test_df.iloc[:]['full_path']